# Output evaluación Leasing X años
## variables
* estimacion_precio_electricidad:dict  (Estimación precios sheet)
* desglose_consumo:dict (Consumo sheet)
* evaluacion_ppa_años:dict (Rentabilidad ESCO sheet)


* O&M_active: bool
* calculo_flujo_cliente:dict   (Cliente sheet)

* * Tasa leasing mensual
* * Plazo
* * Cuota mensual



* Falta que se realice pago PPA FV Flux por X años


In [4]:
import pandas as pd
import json

In [5]:
evaluacion_ppa= {}

In [6]:
with open('../variables/desglose_consumo.json') as archivo_consumo:
    datos_consumo_json=json.load(archivo_consumo)
datos_consumo=pd.DataFrame(datos_consumo_json)


with open('../variables/estimacion_precio_electricidad.json') as archivo_estimacion:
    datos_precio_electricidad_json=json.load(archivo_estimacion)
datos_precio_electricidad=pd.DataFrame(datos_precio_electricidad_json)


with open('../variables/evaluacion_ppa_anos.json') as archivo_evaluacion:
    datos_evaluacion_ppa_anos_json=json.load(archivo_evaluacion)
datos_evaluacion_ppa_anos=pd.DataFrame(datos_evaluacion_ppa_anos_json)


with open('../variables/calculo_flujo_cliente.json') as archivo_flujo_cliente:
    datos_flujo_cliente_json=json.load(archivo_flujo_cliente)
datos_flujo_cliente=pd.DataFrame(datos_flujo_cliente_json)


In [8]:
contador=0

for fecha in datos_consumo:

    #Por fecha
    datos_precio_electricidad_fecha=datos_precio_electricidad[fecha]
    datos_evaluacion_ppa_anos_fecha=datos_evaluacion_ppa_anos[fecha]
    datos_consumo_fecha=datos_consumo[fecha]

    #Variables
    inyeccion=datos_consumo_fecha["Inyeccion"]
    precio_estimado_energia=datos_precio_electricidad_fecha["Precio estm. nominal CLP energia"]
    costo_o_m_admin=datos_evaluacion_ppa_anos_fecha["Ingresos y Egresos"]["O&M y Admin"]
    margen_o_m= datos_flujo_cliente["Calculo flujo cliente"] ["Margen de O&M"]
    margen_o_m_porcentaje=margen_o_m/100
    consumo_distribuidora= datos_consumo_fecha["Consumo distribuidora"]
    precio_estimado_transporte =datos_precio_electricidad_fecha["Precio estm. nominal CLP Transporte"]
    consumo_cliente=datos_consumo_fecha["Consumo cliente"]
    ingreso_venta_energia=datos_evaluacion_ppa_anos_fecha["Ingresos y Egresos"]["Ingreso venta energia PPA"]
    ipc=datos_precio_electricidad_fecha["IPC"]
    ipc_porcentaje=ipc/100


    #Calculos

    ingreso_inyeccion_FV = round(inyeccion * precio_estimado_energia)
    pago_ppa_fv_flux= round((ingreso_venta_energia*(1+ipc_porcentaje))*-1)
    print(pago_ppa_fv_flux)


    o_m_admin_fv=0
    pago_distribuidora= round((consumo_distribuidora *(precio_estimado_transporte + precio_estimado_energia ))*-1)


    pago_anual_energia_fv_negativo= round(ingreso_inyeccion_FV + pago_ppa_fv_flux + pago_distribuidora)

    pago_anual_energia_sin_fv=round(consumo_cliente*(precio_estimado_transporte + precio_estimado_energia))
    pago_anual_energia_con_fv=round(pago_anual_energia_fv_negativo *-1)

    ahorro_anual= pago_anual_energia_sin_fv - pago_anual_energia_con_fv
    if contador==0:
        ingreso_inyeccion_FV=0
        inversion_planta_fv=0
        pago_anual_energia_fv_negativo=0
        pago_anual_energia_con_fv=0
        ahorro_anual=0
        ahorro_acumulado=0
    else:
        fecha_anterior= str(int(fecha) - 1)
        evaluacion_fecha_anterior=evaluacion_ppa[fecha_anterior]
        ahorro_acumulado_anterior= evaluacion_fecha_anterior["Ahorro acumulado"]
        ahorro_acumulado=ahorro_anual + ahorro_acumulado_anterior


    evaluacion_ppa[fecha]=({

        "Ingresos por inyeccion FV": ingreso_inyeccion_FV,
        "Pago PPA FV Flux": pago_ppa_fv_flux,
        "O&M y admin FV":o_m_admin_fv,
        "Pago distribuidora": pago_distribuidora,
        "Pago anual energia con FV-": pago_anual_energia_fv_negativo,
        "Pago anual energia sin FV":pago_anual_energia_sin_fv,
        "Pago anual energia con FV+" : pago_anual_energia_con_fv,
        "Ahorro anual": ahorro_anual,
        "Ahorro acumulado": ahorro_acumulado,
    })
    
    with open('../evaluaciones/ppa/evaluacion.json', 'w') as file:
        json.dump(evaluacion_ppa, file, indent=4)
    
    contador +=1



0
-42041519
-43741933


In [9]:

with open('../evaluaciones/ppa/evaluacion.json') as archivo_evaluacion:
    datos_ppa_json=json.load(archivo_evaluacion)
datos_ppa=pd.DataFrame(datos_ppa_json)

In [10]:
datos_ppa

,2022,2023,2024
Ingresos por inyeccion FV,0,33566064,35259455
Pago PPA FV Flux,0,-42041519,-43741933
O&M y admin FV,0,0,0
Pago distribuidora,0,-29376187,-31013555
Pago anual energia con FV-,0,-37851642,-39496033
Pago anual energia sin FV,0,46661979,49132722
Pago anual energia con FV+,0,37851642,39496033
Ahorro anual,0,8810337,9636689
Ahorro acumulado,0,8810337,18447026
